In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
if not os.path.exists('temp'):
    os.mkdir('temp')

In [3]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews
from langchain_community.agent_toolkits import FileManagementToolkit

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
@tool
def lastest_news(k: int = 5) -> List[Dict[str, str]]:
    """Look up latest news"""

    news_tool = GoogleNews()
    
    return news_tool.search_latest(k=k) 

In [5]:
answer = lastest_news.invoke({'k': 2})

for ans in answer:
    print(ans)

{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFBlSENJVkpsQWNyMnI4bFdabUJ6SDdHSWFtNXVlSmdHOENENVhKazB5YzhfdVhyZFJ4TFZ2OUpYZHZFaXVIYzVIdVRwVWJvUXlMZXJyVWY3U3dNRGxSWEFwdEhUWlhoVmRVWEE?oc=5', 'content': '[속보] 이준석 “후보 단일화, 이번 선거에 없다…김문수로 못 이겨” - 한겨레'}
{'url': 'https://news.google.com/rss/articles/CBMikAFBVV95cUxNY0JvTFRUOWZzSmJCREhUUnlLU0o2LVZCb1UtNGFhdWRWcTRJeGgwRGU4cUxPVExxN0ZZQnItcnNfWm5fTDZVMTZGQVRESUxJRXBKd3JZNGNIU25qYkJmbDNBelU1OHhtLXVJVGJUREdESE5MMnViOVB4UzdFZWc0cHdOMTRWNzNZVkhPVW9aZGLSAaQBQVVfeXFMUEF5UExlWlRqbkxsVllTU0ppb1IzdnZ1WTBlaWdQZUJqdUhvOXc5LWNWNFFVRUNzTnJ5TEVUN05lZ3ZsWjhKS2RNYVRpUC1tbTlCVzVYWlZzSEowRGNwbDJkemxaYy14eDRQbWVIa3Q4c1J3UTF4MTBmYXpjcnZGS2MxZ2s0V0o5QnlETnZXa0JwY1d4UUY2aFB4U3o5QVRzQzVUTVI?oc=5', 'content': '[속보] 경찰, ‘내란 혐의’ 한덕수·최상목 출국금지 - 조선일보'}


In [6]:
working_directory = 'temp'

toolkit = FileManagementToolkit(root_dir=str(working_directory))
tools = toolkit.get_tools()

In [7]:
tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp')]

In [8]:
tools.append(lastest_news)

In [9]:
tools

[CopyFileTool(root_dir='temp'),
 DeleteFileTool(root_dir='temp'),
 FileSearchTool(root_dir='temp'),
 MoveFileTool(root_dir='temp'),
 ReadFileTool(root_dir='temp'),
 WriteFileTool(root_dir='temp'),
 ListDirectoryTool(root_dir='temp'),
 StructuredTool(name='lastest_news', description='Look up latest news', args_schema=<class 'langchain_core.utils.pydantic.lastest_news'>, func=<function lastest_news at 0x000002254E21EF20>)]

In [10]:
for tool in tools:
    print(f'{tool.name} : {tool.description}')

copy_file : Create a copy of a file in a specified location
file_delete : Delete a file
file_search : Recursively search for files in a subdirectory that match the regex pattern
move_file : Move or rename a file from one location to another
read_file : Read file from disk
write_file : Write file to disk
list_directory : List files and directories in a specified folder
lastest_news : Look up latest news


In [11]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini'
)

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system', 
            'You are a helpful assistant. '
            'Make sure to use the `lastest_news` tool to find lastest news. '
            'Make sure to use the `file_managent` tool to manage files. '
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}')
    ]
)

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)      

In [14]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=False, 
    max_iterations=10, 
    max_execution_time=10, 
    handle_parsing_errors=True
)

In [16]:
store = {}

def get_session_history(session_ids):  # session_ids ->'abc123'
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    
    return store[session_ids]

In [18]:
get_session_history('abc123')

InMemoryChatMessageHistory(messages=[])